In [1]:
import numpy as np
import pandas as pd

import _pickle as cPickle
from copy import deepcopy
from datetime import date, datetime, timedelta
import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import pickle
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import time
from tqdm import tqdm

import xgboost
import catboost

In [2]:
leaked_df = pd.read_feather('../input/leak_data.feather')#.rename(columns={'meter_reading': 'leaked_meter_reading'})

In [3]:
# 覚え書き
# 連続で同じ値を取るやつを除去
# ただし、同じ値を取るやつが最小値だった場合は除去しない(電気データの場合、最小値=休みの日とかの可能性があるため)

del_list = list()

for building_id in range(1449):
    leaked_df_gb = leaked_df[leaked_df['building_id'] == building_id].groupby("meter")

    for meter, tmp_df in leaked_df_gb:
        print("building_id: {}, meter: {}".format(building_id, meter))
        data = tmp_df['meter_reading'].values
#         splited_value = np.split(data, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
#         splited_date = np.split(tmp_df.timestamp.values, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
        splited_idx = np.split(tmp_df.index.values, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
        for i, x in enumerate(splited_idx):
            if len(x) > 48:
#                 print("length: {},\t{}-{},\tvalue: {}".format(len(x), x[0], x[-1], splited_value[i][0]))
                del_list.extend(x[1:])
                
                
        print()

del tmp_df, leaked_df_gb

building_id: 0, meter: 0.0

building_id: 1, meter: 0.0

building_id: 2, meter: 0.0

building_id: 3, meter: 0.0

building_id: 4, meter: 0.0

building_id: 5, meter: 0.0

building_id: 6, meter: 0.0

building_id: 7, meter: 0.0

building_id: 7, meter: 1.0

building_id: 8, meter: 0.0

building_id: 9, meter: 0.0

building_id: 9, meter: 1.0

building_id: 10, meter: 0.0

building_id: 11, meter: 0.0

building_id: 12, meter: 0.0

building_id: 13, meter: 0.0

building_id: 13, meter: 1.0

building_id: 14, meter: 0.0

building_id: 14, meter: 1.0

building_id: 15, meter: 0.0

building_id: 15, meter: 1.0

building_id: 16, meter: 0.0

building_id: 17, meter: 0.0

building_id: 18, meter: 0.0

building_id: 19, meter: 0.0

building_id: 20, meter: 0.0

building_id: 21, meter: 0.0

building_id: 22, meter: 0.0

building_id: 23, meter: 0.0

building_id: 24, meter: 0.0

building_id: 25, meter: 0.0

building_id: 26, meter: 0.0

building_id: 27, meter: 0.0

building_id: 28, meter: 0.0

building_id: 28, meter: 1.


building_id: 203, meter: 3.0

building_id: 204, meter: 0.0

building_id: 204, meter: 1.0

building_id: 205, meter: 0.0

building_id: 205, meter: 1.0

building_id: 206, meter: 0.0

building_id: 206, meter: 1.0

building_id: 206, meter: 3.0

building_id: 207, meter: 0.0

building_id: 207, meter: 1.0

building_id: 208, meter: 0.0

building_id: 208, meter: 1.0

building_id: 209, meter: 0.0

building_id: 209, meter: 1.0

building_id: 209, meter: 3.0

building_id: 210, meter: 0.0

building_id: 210, meter: 1.0

building_id: 210, meter: 3.0

building_id: 211, meter: 0.0

building_id: 212, meter: 0.0

building_id: 212, meter: 1.0

building_id: 212, meter: 3.0

building_id: 213, meter: 0.0

building_id: 213, meter: 1.0

building_id: 213, meter: 3.0

building_id: 214, meter: 0.0

building_id: 214, meter: 1.0

building_id: 215, meter: 0.0

building_id: 215, meter: 1.0

building_id: 215, meter: 3.0

building_id: 216, meter: 0.0

building_id: 216, meter: 1.0

building_id: 216, meter: 3.0

building_


building_id: 1333, meter: 1.0

building_id: 1334, meter: 0.0

building_id: 1336, meter: 1.0

building_id: 1336, meter: 2.0

building_id: 1337, meter: 0.0

building_id: 1337, meter: 1.0

building_id: 1337, meter: 2.0

building_id: 1338, meter: 0.0

building_id: 1339, meter: 1.0

building_id: 1339, meter: 2.0

building_id: 1341, meter: 0.0

building_id: 1341, meter: 1.0

building_id: 1341, meter: 2.0

building_id: 1342, meter: 0.0

building_id: 1342, meter: 1.0

building_id: 1342, meter: 2.0

building_id: 1343, meter: 1.0

building_id: 1344, meter: 0.0

building_id: 1345, meter: 0.0

building_id: 1345, meter: 1.0

building_id: 1345, meter: 2.0

building_id: 1346, meter: 0.0

building_id: 1346, meter: 2.0

building_id: 1347, meter: 1.0

building_id: 1347, meter: 2.0

building_id: 1348, meter: 1.0

building_id: 1350, meter: 1.0

building_id: 1350, meter: 2.0

building_id: 1351, meter: 0.0

building_id: 1352, meter: 1.0

building_id: 1353, meter: 0.0

building_id: 1354, meter: 1.0

buildin

In [4]:
len(del_list)

114850

In [5]:
def idx_to_drop(df):
    drop_cols = []
    electric_zero = df[(df['meter']==0)&(df['meter_reading']==0)].index.values.tolist()
    drop_cols.extend(electric_zero)
    not_summer = df[(df['timestamp'].dt.month!=7)&(df['timestamp'].dt.month!=8)]
    not_summer['cumsum'] = not_summer.groupby(['building_id','meter'])['meter_reading'].cumsum()
    not_summer['shifted'] = not_summer.groupby(['building_id','meter'])['cumsum'].shift(48)
    not_summer['difference'] = not_summer['cumsum']-not_summer['shifted']
    steam_zero = not_summer[(not_summer['difference']==0) & (not_summer['meter']==2)].index.values.tolist()
    hotwater_zero = not_summer[(not_summer['difference']==0) & (not_summer['meter']==3)].index.values.tolist()
    drop_cols.extend(steam_zero)
    drop_cols.extend(hotwater_zero)
    del not_summer
    not_winter = leaked_df[(df['timestamp'].dt.month!=12)&(df['timestamp'].dt.month!=1)]
    not_winter['cumsum'] = not_winter.groupby(['building_id','meter'])['meter_reading'].cumsum()
    not_winter['shifted'] = not_winter.groupby(['building_id','meter'])['cumsum'].shift(48)
    not_winter['difference'] = not_winter['cumsum']-not_winter['shifted']
    chilled_zero = not_winter[(not_winter['difference']==0) & (not_winter['meter']==1)].index.values.tolist()
    drop_cols.extend(chilled_zero)
    return drop_cols

del_list.extend(idx_to_drop(leaked_df))

/mnt/dirac/sano/env36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/mnt/dirac/sano/env36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/mnt/dirac/sano/env36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [6]:
len(del_list)

1015415

In [7]:
del_list_new = leaked_df.loc[del_list].index#query('timestamp < 20160901').index

In [8]:
leaked_df = leaked_df.drop(del_list_new).reset_index(drop=True)

In [9]:
with open('../input/leak_data_drop_bad_rows.pkl', 'wb') as f:
    pickle.dump(leaked_df,f)